In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 6 - M. Massot 2020-2021 - Ecole polytechnique
#    ----------   
#    Méthodes de Newton et de la sécante
#    
#    Auteurs : L. Séries et M. Massot - (C) 2021
#   

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Méthodes de Newton et de la sécante

## Itération de la méthode de Newton

On propose de tester la méthode Newton sur la fonction :

$$f(x) = x^3 -2 x -5 $$

In [ ]:
def f(x):
    return x**3 - 2*x - 5

def df(x):
    return 3*x**2 - 2

In [ ]:
def newton(f, df, x0, xstar, tol=1.e-12, nitmax=30):

    # initialisation
    x = np.zeros(nitmax+1)
    err = np.zeros(nitmax+1)
    err_res = np.zeros(nitmax+1)
    diff_x = np.zeros(nitmax)
    x[0] = x0
    err[0] = abs(x0 - xstar)
    err_res[0] = abs(f(x0))

    # iteration de Newton
    for i in range(1, nitmax+1):
        x[i] = x[i-1] - f(x[i-1])/df(x[i-1])
        err[i] = abs(x[i]-xstar)
        err_res[i] = abs(f(x[i]))
        diff_x[i-1] = abs(x[i-1]-x[i])
        print(f"it = {i}, xn = {x[i]:14.8e}, en = {err[i]:14.8e}, |f(xn)| = {err_res[i]:14.8e}, |xn - xn-1| = {diff_x[i-1]:14.8e}" )
        ##if ( f(x[i]) < tol ): break
        if ( err[i] < tol ): break

    return x[0:i+1], err[0:i+1], err_res[0:i+1], diff_x[0:i]

In [ ]:
xsol, err, err_res, diff_x = newton(f, df, x0=4., xstar=2.0945514815423265)

In [ ]:
xmin = 1.9; xmax = 4.1
x = np.linspace(1.9, 4.1, 100)

fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Scatter(x=x, y=f(x), name='f(x)', line_color="cornflowerblue", legendgroup = '1'), row=1, col=1)
fig.add_shape(type="line", x0=xmin, y0=0, x1=xmax, y1=0, row=1, col=1)
for i in range(xsol.size-1):
    fig.add_trace(go.Scatter(x=[xsol[i], xsol[i]], y=[0, f(xsol[i])], showlegend=False,
                             line=dict(color='crimson', width=2, dash='dash')), row=1, col=1)
    fig.add_trace(go.Scatter(x=[xsol[i], xsol[i+1]], y=[f(xsol[i]), 0], showlegend=False,
                             line=dict(color='crimson', width=2, dash='dash')), row=1, col=1)

g = lambda x : x - f(x)/df(x)

fig.add_trace(go.Scatter(x=x, y=g(x), name='g(x) = x - f(x)/df(x)', line_color="green", legendgroup = '2'), row=2, col=1)
fig.add_trace(go.Scatter(x=x, y=x, line_color="black", name='y=x', legendgroup = '2'), row=2, col=1)
fig.add_shape(type="line", x0=xmin, y0=0, x1=xmax, y1=0, row=2, col=1)

for i in range(xsol.size-1):
    fig.add_trace(go.Scatter(x=[xsol[i], xsol[i]], y=[0, g(xsol[i])], showlegend=False,
                              line=dict(color='crimson', width=2, dash='dash')), row=2, col=1)
    fig.add_trace(go.Scatter(x=[xsol[i], xsol[i+1]], y=[g(xsol[i]), xsol[i+1]], showlegend=False,
                             line=dict(color='crimson', width=2, dash='dash')), row=2, col=1)
    fig.add_trace(go.Scatter(x=[xsol[i+1], xsol[i+1]], y=[xsol[i+1], g(xsol[i+1])], showlegend=False,
                             line=dict(color='crimson', width=2, dash='dash')), row=2, col=1)

fig.update_xaxes(range=[1.9, 4.1], tickmode = 'array', tickvals=xsol[:4], tickformat='.2f')    
fig.update_yaxes(range=[-0.1, 4.1], row=1, col=2)
fig.update_layout(title="Itérations de Newton", height=1000, legend_tracegroupgap=450)    
fig.show()

## Convergence de la méthode de Newton

In [ ]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=np.arange(err.size), y=err, name='erreur', mode='markers')) 
fig1.add_trace(go.Scatter(x=np.arange(err_res.size), y=err_res, name='erreur residuelle', mode='markers')) 
fig1.add_trace(go.Scatter(x=np.arange(1, err.size), y=diff_x[:], name='différence des itérés', mode='markers')) 
fig1.update_xaxes(title='Itération')    
fig1.update_yaxes(type="log", exponentformat='e', title="Erreur")    
fig1.update_layout(title="Historique de convergence")    
fig1.show()

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=err[:-1:], y=err[1:], name='erreur', mode='markers')) 
fig2.add_trace(go.Scatter(x=err[:-1], y=err[:-1], name='droite de pente -1', mode='lines', line_dash='dot')) 
fig2.add_trace(go.Scatter(x=err[:-1], y=err[:-1]*err[:-1], name='droite de pente -2', mode='lines', line_dash='dot')) 
fig2.update_xaxes(type="log", exponentformat='e', title="Erreur à l'itération k")    
fig2.update_yaxes(type="log", exponentformat='e', title="Erreur à l'itération k+1") 
fig2.update_layout(title="Ordre de convergence")    
fig2.show()

# Méthode de la sécante

On propose de tester la méthode de la sécante sur la même fonction :

$$f(x) = x^3 -2 x -5 $$

In [ ]:
def secante(f, x0, x1, xstar, tol=1.e-12, nitmax=50):
   
    # initialisation
    x = np.zeros(nitmax+2)
    err = np.zeros(nitmax+2)
    err_res = np.zeros(nitmax+2)
    diff_x = np.zeros(nitmax+1)
    x[0] = x0
    x[1] = x1
    err[0] = abs(x0 - xstar)
    err[1] = abs(x1 - xstar)
    err_res[0] = abs(f(x0))
    err_res[1] = abs(f(x1))
    diff_x[0] = abs(x1-x0)
    
    # iteration de la méthode de la sécante        
    for i in range(2, nitmax+2):
        x[i] = x[i-1] - ((x[i-1] - x[i-2])/(f(x[i-1]) - f(x[i-2])))*f(x[i-1])
        err[i] = abs(x[i]-xstar)
        err_res[i] = abs(f(x[i]))
        diff_x[i-1] = abs(x[i-1]-x[i])
        print(f"it = {i-1}, xn = {x[i]:16.8e}, en = {err[i]:16.8e}, |f(xn)| = {err_res[i]:16.8e}, |xn - xn-1| = {diff_x[i-1]:16.8e}" )
        #print(i, x[i], err_res[i], err[i])
        ##if ( f(x[i]) < tol ): break
        if ( err[i] < tol ): break

    return x[0:i+1], err[0:i+1], err_res[0:i+1], diff_x[0:i]

## Itération de la méthode de la sécante

In [ ]:
xsol, err, err_res, diff_x = secante(f, x0=4, x1=3.8, xstar=2.0945514815423265)

In [ ]:
xmin = 1.9; xmax = 4.1
x = np.linspace(1.9, 4.1, 100)

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=f(x), name='f(x)', line_color="cornflowerblue"))
fig.add_shape(type="line", x0=xmin, y0=0, x1=xmax, y1=0)
fig.add_trace(go.Scatter(x=xsol, y=f(xsol), showlegend=False, mode='markers'))
for i in range(xsol.size-2):
    fig.add_trace(go.Scatter(x=[xsol[i], xsol[i+1], xsol[i+2]], y=[f(xsol[i]), f(xsol[i+1]), 0], showlegend=False,
                             line=dict(color='crimson', width=2, dash='dash')))
    fig.add_trace(go.Scatter(x=[xsol[i+2], xsol[i+2]], y=[0, f(xsol[i+2])], showlegend=False,
                             line=dict(color='crimson', width=2, dash='dash')))

fig.update_layout(title="Itérations de la méthode de la sécante")    
fig.show()                             

## Convergence de la méthode de la sécante

In [ ]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=np.arange(err.size), y=err, name='erreur', mode='markers')) 
fig1.add_trace(go.Scatter(x=np.arange(err_res.size), y=err_res, name='erreur residuelle', mode='markers')) 
fig1.add_trace(go.Scatter(x=np.arange(1, err.size), y=diff_x[:], name='différence des itérés', mode='markers')) 
fig1.update_xaxes(title='Itération')    
fig1.update_yaxes(type="log", exponentformat='e', title="Erreur")    
fig1.update_layout(title="Historique de convergence")    
fig1.show()

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=err[:-1:], y=err[1:], name='erreur', mode='markers')) 
fig2.add_trace(go.Scatter(x=err[:-1], y=err[:-1], name='droite de pente -1', mode='lines', line_dash='dot')) 
fig2.add_trace(go.Scatter(x=err[:-1], y=err[:-1]*err[:-1], name='droite de pente -2', mode='lines', line_dash='dot')) 
fig2.update_xaxes(type="log", exponentformat='e', title="Erreur à l'itération k")    
fig2.update_yaxes(type="log", exponentformat='e', title="Erreur à l'itération k+1") 
fig2.update_layout(title="Ordre de convergence")    
fig2.show()